In [ ]:
from ukkonen import SuffixTree, Node
from suffix_array import SuffixArray

In [2]:
def longest_common_substring_sa(str1: str, str2: str) -> str:
    """
    Find the longest common substring of two strings using a suffix array.
 
    Args:
        str1: First string
        str2: Second string
 
    Returns:
        The longest common substring
    """
    # Concatenate the strings with a unique separator
    combined = str1 + "#" + str2 + "$"
    seperator_index = len(str1)
 
    sa = SuffixArray(combined)

    n = len(sa.suffixes)
    lcp = [0] * (n-1)
    rank = [0] * n
    for i in range(n):
        rank[sa.suffixes[i]] = i

    k = 0
    for i in range(n):
        if rank[i] == n - 1:
            k = 0
            continue

        j = sa.suffixes[rank[i] + 1]
        while i + k < n and j + k < n and sa.text[i + k] == sa.text[j + k]:
            k += 1

        lcp[rank[i]] = k
        if k > 0:
            k -= 1
    
    max_len = 0
    position = 0
    for i in range(1, n):
        s1 = sa.suffixes[i]
        s2 = sa.suffixes[i - 1]

        if (s1 < seperator_index) != (s2 < seperator_index):
            if lcp[i - 1] > max_len:
                max_len = lcp[i - 1]
                position = s1

    return combined[position : position + max_len]

In [ ]:
def longest_common_substring_st(str1: str, str2: str) -> str:
    """
    Find the longest common substring of two strings using a suffix tree.
 
    Args:
        str1: First string
        str2: Second string
 
    Returns:
        The longest common substring
    """
    # Concatenate the strings with a unique separator
    combined = str1 + "#" + str2 + "$"
    seperator_index = len(str1)
    longest_substring = ""
 
    # Build a suffix tree for the combined string
    st = SuffixTree(combined)

    # Traverse the tree to find the longest path that occurs in both strings
    def DFS(node : Node, path : list):
        nonlocal longest_substring
        bits = set()

        if not node.children:
            if node.id < seperator_index:
                bits.add(0)
            elif node.id > seperator_index:
                bits.add(1)
            return bits
        
        for child in node.children.values():
            edge_end = child.end.value if hasattr(child.end, 'value') else child.end
            edge_text = st.text[child.start : edge_end]

            bits.update(DFS(child, path + [edge_text]))

        if 0 in bits and 1 in bits:
            substring = "".join(path)
            if len(substring) > len(longest_substring):
                longest_substring = substring

        return bits
    
    DFS(st.root, [])
    return longest_substring